In [1]:
import pandas as pd
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
%matplotlib inline

font_dirs = ['./font'] # 폰트경로
font_files = fm.findSystemFonts(fontpaths=font_dirs)
for font_file in font_files:
    fm.fontManager.addfont(font_file)
plt.rcParams['font.family'] = 'Noto Sans CJK KR'
%config InlineBackend.figure_format = 'retina'

In [2]:
from glob import glob

In [3]:
full_files = glob('./data/*_full.xlsx')
num_files = glob('./data/*_number.xlsx')

In [4]:
full_files = sorted(full_files)

In [5]:
num_files = sorted(num_files)

In [6]:
sido_list = ['서울', '부산', '대구', '인천', '광주', '대전', '울산', '세종', '경기',
               '강원', '충북', '충남', '전북', '전남', '경북', '경남', '제주']

In [7]:
def calculate_rate(d):
    full, number =d['입학정원'],d['입학자수']
#     if full==0:
#         return 
    return number/(full+1) * 100
def create_new_index(d):
    sido, sigungu = d['시도'],d['행정구']
    return str(sido)+'|' + str(sigungu)
def split_hjgu(d):
    d = d['행정구역']
    if type(d) == float:
        return None
    return d.split()[-1]

def split_hjgu_new(d):
    d = d['행정구']
    if type(d) == float:
        return None
    return d.split()[-1]

def fill_hyphen(d):
    if d == '-':
        return 0
    return d

In [8]:
result = []
for i in range(len(num_files)):
    year = num_files[i].split('/')[-1].split('_')[0]
    df_number = pd.read_excel(num_files[i])
    df_full = pd.read_excel(full_files[i])

    if '행정구' not in df_number.columns:
        df_number['행정구'] = df_number.apply(split_hjgu, axis=1)
    if '행정구'  not in df_full.columns:
        df_full['행정구'] = df_full.apply(split_hjgu, axis=1)

    df_number['행정구역'] = df_number['행정구']
    df_full['행정구역'] = df_full['행정구']
    df_number = df_number.applymap(fill_hyphen)
    df_full = df_full.applymap(fill_hyphen)
    if '입학정원' not in df_full.columns:
        df_full['입학정원'] = df_full['전체']
        df_number['입학자수'] = df_number['전체']
    df_list = list()
    df_full['rename_index']= df_full.apply(create_new_index,axis=1)
    df_number['rename_index']= df_number.apply(create_new_index,axis=1)
    
    df1=df_full[['rename_index','시도','행정구역','입학정원']]
    df2=df_number[['rename_index','시도','행정구역','입학자수']]
    print(df1.columns,df2.columns)
    for isido in sido_list:
        tmp_df=pd.merge(df1[(df1['시도']==isido)\
                            
                            &(df1['행정구역']!='총계')\
                            &(df1['행정구역']!='행정구역')][['시도','행정구역','입학정원']], 
                        df2[(df2['시도']==isido)\
                            
                            &(df2['행정구역']!='총계')\
                            &(df2['행정구역']!='행정구역')][['시도','행정구역','입학자수']], 
                        how='outer', on=None)
        
        tmp_df.columns=['시도','행정구','입학정원','입학자수']
        tmp_df['index'] = tmp_df.apply(create_new_index, axis=1)
        
        tmp_df = tmp_df.set_index('index')
        df_list.append(tmp_df)
        
    df=pd.DataFrame()
    for idf in df_list:
        df = df.append(idf)
    df[year+'|rate']=df.apply(calculate_rate,axis=1)
    df.columns=['시도','행정구역',year+'|입학정원',year+'|입학자수',year+'|rate']
    df = df[['시도',year+'|입학정원',year+'|입학자수',year+'|rate']]
    result.append(df)

/Users/jmchoi/opt/anaconda3/envs/soprise/lib/python3.7/site-packages/openpyxl/styles/stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Index(['rename_index', '시도', '행정구역', '입학정원'], dtype='object') Index(['rename_index', '시도', '행정구역', '입학자수'], dtype='object')
Index(['rename_index', '시도', '행정구역', '입학정원'], dtype='object') Index(['rename_index', '시도', '행정구역', '입학자수'], dtype='object')
Index(['rename_index', '시도', '행정구역', '입학정원'], dtype='object') Index(['rename_index', '시도', '행정구역', '입학자수'], dtype='object')
Index(['rename_index', '시도', '행정구역', '입학정원'], dtype='object') Index(['rename_index', '시도', '행정구역', '입학자수'], dtype='object')
Index(['rename_index', '시도', '행정구역', '입학정원'], dtype='object') Index(['rename_index', '시도', '행정구역', '입학자수'], dtype='object')


In [9]:
result[0].iloc[20:30]

,시도,2016|입학정원,2016|입학자수,2016|rate
index,,,,
서울|영등포구,서울,180,90,49.723757
서울|용산구,서울,4223,4288,101.515152
서울|은평구,서울,268,221,82.156134
서울|종로구,서울,75816,36718,48.429772
서울|중구,서울,7148,7750,108.406770
서울|중랑구,서울,2606,3019,115.803606
부산|계,부산,54476,58479,107.346220
부산|금정구,부산,11088,11709,105.591126
부산|남구,부산,10204,10834,106.163645


In [10]:
def filter_total_data(d):
    d=d['index']
    first, second = d.split('|')
    if first =="세종" and second=="계":
        return True
    elif second=="계":
        return False
    else:
        return True
df = result[0].reset_index()
df['filter']=df.apply(filter_total_data, axis=1)

In [11]:
df[df['filter']==True].iloc[60:70]

,index,시도,2016|입학정원,2016|입학자수,2016|rate,filter
67,울산|동구,울산,1800,2042,113.381455,True
68,울산|울주군,울산,1771,1868,105.417607,True
69,울산|중구,울산,405,484,119.211823,True
70,세종|계,세종,4832,5383,111.380095,True
71,세종|세종,세종,4832,5383,111.380095,True
73,경기|가평군,경기,38,36,92.307692,True
74,경기|고양시,경기,1384,1487,107.364621,True
75,경기|과천시,경기,40,15,36.585366,True
76,경기|광주시,경기,3396,2973,87.518399,True
77,경기|군포시,경기,953,1002,105.031447,True


In [12]:
df1=pd.merge(left = result[0] , right = result[1], how = "outer", on = 'index')
df2=pd.merge(left = result[2] , right = result[3], how = "outer", on = 'index')
df3 = pd.merge(left = df1, right=df2, how="outer", on="index")
final_df = pd.merge(left = df3, right=result[4], how="outer", on="index")

In [13]:
final_df.head()

,시도_x_x,2016|입학정원,2016|입학자수,2016|rate,시도_y_x,2017|입학정원,2017|입학자수,2017|rate,시도_x_y,2018|입학정원,2018|입학자수,2018|rate,시도_y_y,2019|입학정원,2019|입학자수,2019|rate,시도,2020|입학정원,2020|입학자수,2020|rate
index,,,,,,,,,,,,,,,,,,,,
서울|계,서울,233203,207113,88.811941,서울,232436,202310,87.038638,서울,230041,202833,88.172160,서울,227869,202194,88.732172,서울,NaN,204510,NaN
서울|강남구,서울,440,457,103.628118,서울,449,456,101.333333,서울,449,439,97.555556,서울,554,619,111.531532,서울,554,598,107.747748
서울|강동구,서울,50,48,94.117647,서울,50,86,168.627451,서울,50,20,39.215686,서울,50,35,68.627451,서울,50,27,52.941176
서울|강북구,서울,3621,3474,95.913860,서울,3561,2843,79.814711,서울,3561,3341,93.795620,서울,3561,3278,92.026951,서울,3561,4204,118.023582
서울|강서구,서울,742,831,111.843876,서울,734,850,115.646259,서울,3701,3214,86.817936,서울,3701,3055,82.522961,서울,3701,2684,72.501351


In [14]:
final_df =final_df.reset_index()

In [15]:
def filter_total_data(d):
    d=d['index']
    first, second = d.split('|')
    if first =="세종" and second=="계":
        return True
    elif second=="계":
        return None
    elif second=='nan':
        return None
    else:
        return True
final_df['filter'] = final_df.apply(filter_total_data,axis=1)

In [16]:
df = final_df[final_df['filter']==True]

In [17]:
df.head()

,index,시도_x_x,2016|입학정원,2016|입학자수,2016|rate,시도_y_x,2017|입학정원,2017|입학자수,2017|rate,시도_x_y,...,2018|rate,시도_y_y,2019|입학정원,2019|입학자수,2019|rate,시도,2020|입학정원,2020|입학자수,2020|rate,filter
1,서울|강남구,서울,440,457,103.628118,서울,449,456,101.333333,서울,...,97.555556,서울,554,619,111.531532,서울,554,598,107.747748,True
2,서울|강동구,서울,50,48,94.117647,서울,50,86,168.627451,서울,...,39.215686,서울,50,35,68.627451,서울,50,27,52.941176,True
3,서울|강북구,서울,3621,3474,95.913860,서울,3561,2843,79.814711,서울,...,93.795620,서울,3561,3278,92.026951,서울,3561,4204,118.023582,True
4,서울|강서구,서울,742,831,111.843876,서울,734,850,115.646259,서울,...,86.817936,서울,3701,3055,82.522961,서울,3701,2684,72.501351,True
5,서울|관악구,서울,8453,8729,103.252898,서울,8447,8612,101.941288,서울,...,100.023552,서울,8471,8343,98.477337,서울,8457,8541,100.981319,True


In [18]:
df=df[['index','시도_x_x',
    '2016|입학정원','2016|입학자수','2016|rate',
    '2017|입학정원','2017|입학자수','2017|rate',
    '2018|입학정원','2018|입학자수','2018|rate',
    '2019|입학정원','2019|입학자수','2019|rate',
    '2020|입학정원','2020|입학자수','2020|rate', 'filter'
   ]]
df = df.set_index('index')

In [19]:
df.columns=['시도']+df.columns[1:].tolist()
df[df.columns[:-1]].head()

,시도,2016|입학정원,2016|입학자수,2016|rate,2017|입학정원,2017|입학자수,2017|rate,2018|입학정원,2018|입학자수,2018|rate,2019|입학정원,2019|입학자수,2019|rate,2020|입학정원,2020|입학자수,2020|rate
index,,,,,,,,,,,,,,,,
서울|강남구,서울,440,457,103.628118,449,456,101.333333,449,439,97.555556,554,619,111.531532,554,598,107.747748
서울|강동구,서울,50,48,94.117647,50,86,168.627451,50,20,39.215686,50,35,68.627451,50,27,52.941176
서울|강북구,서울,3621,3474,95.913860,3561,2843,79.814711,3561,3341,93.795620,3561,3278,92.026951,3561,4204,118.023582
서울|강서구,서울,742,831,111.843876,734,850,115.646259,3701,3214,86.817936,3701,3055,82.522961,3701,2684,72.501351
서울|관악구,서울,8453,8729,103.252898,8447,8612,101.941288,8491,8494,100.023552,8471,8343,98.477337,8457,8541,100.981319


In [20]:
df.iloc[63:69]

,시도,2016|입학정원,2016|입학자수,2016|rate,2017|입학정원,2017|입학자수,2017|rate,2018|입학정원,2018|입학자수,2018|rate,2019|입학정원,2019|입학자수,2019|rate,2020|입학정원,2020|입학자수,2020|rate,filter
index,,,,,,,,,,,,,,,,,
세종|계,세종,4832,5383,111.380095,4762,5290,111.064455,4603,5119,111.185925,4603,5177,112.445699,NaN,5141,NaN,True
세종|계,세종,4832,5383,111.380095,4762,5290,111.064455,4603,5119,111.185925,4603,5177,112.445699,NaN,5141,NaN,True
세종|계,세종,4832,5383,111.380095,4762,5290,111.064455,4603,5119,111.185925,4603,5177,112.445699,NaN,5141,NaN,True
세종|계,세종,4832,5383,111.380095,4762,5290,111.064455,4603,5119,111.185925,4603,5177,112.445699,NaN,5141,NaN,True
세종|세종,세종,4832,5383,111.380095,4762,5290,111.064455,NaN,NaN,NaN,NaN,NaN,NaN,4588,5141,112.028764,True
경기|가평군,경기,38,36,92.307692,38,35,89.743590,38,23,58.974359,38,25,64.102564,38,32,82.051282,True


In [21]:
df.loc['세종|세종','2018|입학정원':'2019|rate'] = [4603,5119,
                                           111.185925,
                                           4603,5177,
                                           112.445699]

In [22]:
df.iloc[63:69]

,시도,2016|입학정원,2016|입학자수,2016|rate,2017|입학정원,2017|입학자수,2017|rate,2018|입학정원,2018|입학자수,2018|rate,2019|입학정원,2019|입학자수,2019|rate,2020|입학정원,2020|입학자수,2020|rate,filter
index,,,,,,,,,,,,,,,,,
세종|계,세종,4832,5383,111.380095,4762,5290,111.064455,4603,5119,111.185925,4603,5177,112.445699,NaN,5141,NaN,True
세종|계,세종,4832,5383,111.380095,4762,5290,111.064455,4603,5119,111.185925,4603,5177,112.445699,NaN,5141,NaN,True
세종|계,세종,4832,5383,111.380095,4762,5290,111.064455,4603,5119,111.185925,4603,5177,112.445699,NaN,5141,NaN,True
세종|계,세종,4832,5383,111.380095,4762,5290,111.064455,4603,5119,111.185925,4603,5177,112.445699,NaN,5141,NaN,True
세종|세종,세종,4832,5383,111.380095,4762,5290,111.064455,4603,5119,111.185925,4603,5177,112.445699,4588,5141,112.028764,True
경기|가평군,경기,38,36,92.307692,38,35,89.743590,38,23,58.974359,38,25,64.102564,38,32,82.051282,True


In [23]:
df = df.drop(['세종|계']) # 필요없는 데이터 제거

In [24]:
df.iloc[60:65]

,시도,2016|입학정원,2016|입학자수,2016|rate,2017|입학정원,2017|입학자수,2017|rate,2018|입학정원,2018|입학자수,2018|rate,2019|입학정원,2019|입학자수,2019|rate,2020|입학정원,2020|입학자수,2020|rate,filter
index,,,,,,,,,,,,,,,,,
울산|동구,울산,1800,2042,113.381455,1768,2009,113.566987,1763,2041,115.702948,1803,2181,120.898004,1743,2237,128.268349,True
울산|울주군,울산,1771,1868,105.417607,1755,1880,107.061503,1755,1823,103.815490,1755,1875,106.776765,1663,1765,106.069712,True
울산|중구,울산,405,484,119.211823,405,476,117.241379,405,475,116.995074,355,401,112.640449,325,377,115.644172,True
세종|세종,세종,4832,5383,111.380095,4762,5290,111.064455,4603,5119,111.185925,4603,5177,112.445699,4588,5141,112.028764,True
경기|가평군,경기,38,36,92.307692,38,35,89.743590,38,23,58.974359,38,25,64.102564,38,32,82.051282,True


In [25]:
df = df[df.columns[0:-1]].reset_index()
df.head()

,index,시도,2016|입학정원,2016|입학자수,2016|rate,2017|입학정원,2017|입학자수,2017|rate,2018|입학정원,2018|입학자수,2018|rate,2019|입학정원,2019|입학자수,2019|rate,2020|입학정원,2020|입학자수,2020|rate
0,서울|강남구,서울,440,457,103.628118,449,456,101.333333,449,439,97.555556,554,619,111.531532,554,598,107.747748
1,서울|강동구,서울,50,48,94.117647,50,86,168.627451,50,20,39.215686,50,35,68.627451,50,27,52.941176
2,서울|강북구,서울,3621,3474,95.913860,3561,2843,79.814711,3561,3341,93.795620,3561,3278,92.026951,3561,4204,118.023582
3,서울|강서구,서울,742,831,111.843876,734,850,115.646259,3701,3214,86.817936,3701,3055,82.522961,3701,2684,72.501351
4,서울|관악구,서울,8453,8729,103.252898,8447,8612,101.941288,8491,8494,100.023552,8471,8343,98.477337,8457,8541,100.981319


In [26]:
df.to_json('./data/data.json', orient='records',force_ascii=False)
